the data is located /Users/deepakmahtani/.kaggle/competitions/titanic

First thing to do is to import some standard packages

In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

Now lest read in the training data and test data

In [2]:
raw_training=pd.read_csv('/Users/deepakmahtani/.kaggle/competitions/titanic/train.csv')
raw_test=pd.read_csv('/Users/deepakmahtani/.kaggle/competitions/titanic/test.csv')

In [9]:
raw_training[['Cabin','Pclass']][raw_training['Cabin'].notnull()]

,Cabin,Pclass
1,C85,1
3,C123,1
6,E46,1
10,G6,3
11,C103,1
21,D56,2
23,A6,1
27,C23 C25 C27,1
31,B78,1
52,D33,1


After chatting with John the columns we are going to use are Survived, pclass, sex, age,SibSp and Parch

In [3]:
training_to_use=raw_training[['PassengerId', 'Survived', 'Pclass','Sex','Age','SibSp', 'Parch']]


test_to_use=raw_test[['PassengerId', 'Pclass','Sex','Age','SibSp', 'Parch']]

I know that the gender column is very important but we need to change this from male and female to a numeric value so lets change this in both the training and text set

In [4]:
training_to_use.replace(to_replace='male', value=0, inplace=True)
training_to_use.replace(to_replace='female', value=1, inplace=True)

test_to_use.replace(to_replace='male', value=0, inplace=True)
test_to_use.replace(to_replace='female', value=1, inplace=True)

/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/dee

now lets replace the missing age values in the data with the median value. The reason we have chosen the median age and not the mean age is beacuse it is less affected by outliers

In [5]:
median_age_training=np.median(training_to_use[pd.notnull(training_to_use['Age'])]['Age'])

median_age_test=np.median(test_to_use[pd.notnull(test_to_use['Age'])]['Age'])

print("median age for the training set", median_age_training)
print("median age for the test set", median_age_test)

median age for the training set 28.0
median age for the test set 27.0


In [6]:
clean_age_train=training_to_use['Age'].fillna(median_age_training)
clean_age_test=test_to_use['Age'].fillna(median_age_test)

training_to_use['clean_age']=clean_age_train
test_to_use['clean_age']=clean_age_test

/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Now lets creat the two data frames to be used for the training and testing of the model

In [7]:
clean_data_training=training_to_use[['PassengerId','Survived', 'Pclass', 'Sex',
                                     'SibSp', 'Parch','clean_age']]

clean_data_test=test_to_use[['PassengerId', 'Pclass', 'Sex','SibSp', 'Parch', 'clean_age']]

Make a train test split of the training set for the grid search

In [8]:
X_grid=clean_data_training[['Pclass', 'Sex','SibSp', 'Parch','clean_age']]
y_grid=clean_data_training['Survived']

X_train, X_test, y_train, y_test = train_test_split(X_grid, y_grid, test_size=0.30)

Next I will initiate the random forrest object

In [9]:
forrest_classifier_grid_search=RandomForestClassifier()

In [10]:
# Number of trees in random forest
#n_estimators = [1,10,24,50]
n_estimators = [10,15,20,24]

# Number of features to consider at every split
#max_features = ['sqrt','auto','log2']

# Maximum number of levels in tree
#max_depth = [1,3,6,10,20,50]

# Minimum number of samples required to split a node
#min_samples_split = [2,4,10,20,50,100]
min_samples_split = [2,4,10,20,30,40,50,60,70,80,90,100]

# Minimum number of samples required at each leaf node
#min_samples_leaf = [int(x) for x in np.linspace(2, 10, num = 8)]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
search_params = {'n_estimators': n_estimators,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap}

In [11]:
grid_search=GridSearchCV(forrest_classifier_grid_search, search_params)

In [12]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 15, 20, 24], 'min_samples_split': [2, 4, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [13]:
print('The best Params to use are', grid_search.best_params_ )

print('Which gives a best score on the training set of',grid_search.best_score_)

The best Params to use are {'bootstrap': False, 'min_samples_split': 70, 'n_estimators': 15}
Which gives a best score on the training set of 0.8250401284109149


The best Params to use are {'bootstrap': True, 'min_samples_split': 2, 'n_estimators': 24}

The best Params to use are {'bootstrap': False, 'min_samples_split': 100, 'n_estimators':24}

The best Params to use are {'bootstrap': True, 'min_samples_split': 50, 'n_estimators':10}

The best Params to use are {'bootstrap': False, 'min_samples_split': 50, 'n_estimators':24}

The best Params to use are {'bootstrap': True, 'min_samples_split': 10, 'n_estimators':10}

it was jumping around a lot min sample split between 2-100 and n_estimators between 10 and 24 so narrowed down th range of these

The best Params to use are {'bootstrap': True, 'min_samples_split': 20, 'n_estimators': 10}<-going to go with this one

Which gives a best score on the training set of 0.8218298555377207

The best Params to use are {'bootstrap': True, 'min_samples_split': 50, 'n_estimators': 24}
Which gives a best score on the training set of 0.8138041733547352

The best Params to use are {'bootstrap': True, 'min_samples_split': 30, 'n_estimators': 24}
Which gives a best score on the training set of 0.8218298555377207



so now train a model using the best params and test using the test data

In [14]:
forrest_classifier_best_params=RandomForestClassifier(bootstrap=True,min_samples_split=20,n_estimators=10)

In [15]:
forrest_classifier_best_params.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
print('this gives a best score of',forrest_classifier_best_params.score(X_test,y_test))

this gives a best score of 0.8022388059701493


so now lets see how well it does on the  real test set, first lets fit the model on all the training data with the best params

In [17]:
forrest_classifier_test_data=RandomForestClassifier(bootstrap=True,min_samples_split=20,n_estimators=10)

In [18]:
X=clean_data_training[['Pclass', 'Sex','SibSp', 'Parch','clean_age']]
y=clean_data_training['Survived']


forrest_classifier_test_data.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

now lets creat the training data on which to make the predictions

In [19]:
X_test=clean_data_test[['Pclass', 'Sex','SibSp', 'Parch','clean_age']]

now lets use the model to make the predictions

In [20]:
predicts=forrest_classifier_test_data.predict(X_test)

creat a data frame to upload to kaggle to see how well it performs

In [ ]:
results=clean_data_test[['PassengerId']]
results['Survived']=predicts

In [ ]:
results.to_csv('random_forrest_optimised.csv',index=False)

My previous modle gave me a score of 0.65071, and this model gave 0.64593 which was not better but this was when I ran it the first time I got best params of bootstrap=False, min_samples_split=50, n_estimators=10. When I did the grid search again I got best params of bootstrap = True, min_samples_split=50, n_estimators = 50

using bootstrap=True,min_samples_split=20,n_estimators=10 which gave 0.65071 which is equal with how good I did before so need to either use more features/do more feature engineering or try a different algorithm